In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot

from tqdm.notebook import tqdm


# Loading File

In [ ]:
f = uproot.open("data_files/output_osc_mc_detail_1.root")


In [ ]:
# from data_files/CombinedFunctions_from_Fortran/CombinedTypes.h
particle_types = {
    1: "gamma",
    2: "positron",
    3: "electron",
    4: "neutrino",
    5: "mu+",
    6: "mu-",
    7: "pi0",
    8: "pi+",
    9: "pi-",
    10: "K0L",
    11: "K+",
    12: "K-",
    13: "neutron",
    14: "proton",
    15: "anti-proton",
    16: "K0S",
    17: "eta",
    18: "lambda",
    19: "sigma+",
    20: "sigma0",
    21: "sigma-",
    22: "xi0",
    23: "xi-",
    24: "omega-",
    25: "anti-neutron",
    26: "anti-lambda",
    27: "anti-sigma-",
    28: "anti-sigma0",
    29: "anti-sigma+",
    30: "anti-xi0",
    31: "anti-xi+",
    32: "anti-omega+",
    45: "deuteron",
    46: "triton",
    47: "alpha",
    48: "geantino",
    49: "He3",
    50: "Cerenkov"
}


In [ ]:
df = f["MiniBooNE_CCQE"].arrays(["EventNumber", "Weight", "PassOsc", "RecoEnuQE", "Energy", "CosTheta", "TrueEnergy", "NuType", "NFSP", "FSPType", "MomX", "MomY", "MomZ", "MomT"], library="pd")

# from data_files/SomeMiniBooNEDetails_on_files_v2.pdf
nu_types = {
    1: "numu",
    2: "numubar",
    3: "nue",
    4: "nuebar"
}
df["nu_type"] = df["NuType"].apply(lambda x: nu_types[x])
df.drop(columns=["NuType"], inplace=True)

df

# 2 shower 0 track truth cut

In [ ]:
particle_codes = df["FSPType"].to_numpy()
particle_MomXs = df["MomX"].to_numpy()
particle_MomYs = df["MomY"].to_numpy()
particle_MomZs = df["MomZ"].to_numpy()
particle_MomTs = df["MomT"].to_numpy()

shower_threshold = 100
track_threshold = 1000

num_true_HE_showers = []
num_true_HE_tracks = []
for i in tqdm(range(len(particle_codes))):
    curr_num_showers = 0
    curr_num_HE_showers = 0
    curr_num_HE_tracks = 0
    curr_particle_codes = particle_codes[i]
    curr_MomTs = particle_MomTs[i]
    for j in range(len(curr_particle_codes)):
        curr_particle_type = particle_types[curr_particle_codes[j]]
        curr_particle_energy = curr_MomTs[j] * 1000

        if (curr_particle_type == "gamma" or curr_particle_type == "electron" or curr_particle_type == "positron") and curr_particle_energy > shower_threshold:
            curr_num_HE_showers += 1

        if (curr_particle_type == "proton" or curr_particle_type == "mu+" or curr_particle_type == "mu-" or curr_particle_type == "pi+" or curr_particle_type == "pi-") and curr_particle_energy > track_threshold:
            curr_num_HE_tracks += 1

    num_true_HE_showers.append(curr_num_HE_showers)
    num_true_HE_tracks.append(curr_num_HE_tracks)

df["num_true_HE_showers"] = num_true_HE_showers
df["num_true_HE_tracks"] = num_true_HE_tracks


In [ ]:
df

In [ ]:
df.query("PassOsc==True")

# True Quantities for two showers

In [ ]:
two_HE_shower_0_HE_track_df = df.query("num_true_HE_showers == 2 and num_true_HE_tracks == 0")

particle_codes = two_HE_shower_0_HE_track_df["FSPType"].to_numpy()
particle_MomXs = two_HE_shower_0_HE_track_df["MomX"].to_numpy()
particle_MomYs = two_HE_shower_0_HE_track_df["MomY"].to_numpy()
particle_MomZs = two_HE_shower_0_HE_track_df["MomZ"].to_numpy()
particle_MomTs = two_HE_shower_0_HE_track_df["MomT"].to_numpy()

leading_energies = []
subleading_energies = []
opening_angles = []
total_momentum_beam_angles = []

for i in tqdm(range(len(particle_codes))):

    curr_leading_energy = -np.inf
    curr_subleading_energy = -np.inf
    curr_leading_momentum = np.array([np.nan, np.nan, np.nan])
    curr_subleading_momentum = np.array([np.nan, np.nan, np.nan])

    curr_particle_codes = particle_codes[i]
    curr_MomXs = particle_MomXs[i]
    curr_MomYs = particle_MomYs[i]
    curr_MomZs = particle_MomZs[i]
    curr_MomTs = particle_MomTs[i]

    for j in range(len(curr_particle_codes)):
        curr_particle_type = particle_types[curr_particle_codes[j]]
        curr_particle_energy = curr_MomTs[j] * 1000
        curr_particle_momentum = np.array([curr_MomXs[j], curr_MomYs[j], curr_MomZs[j]])
        if curr_particle_type == "gamma" or curr_particle_type == "electron" or curr_particle_type == "positron" and curr_particle_energy > 20:
            if curr_particle_energy > curr_leading_energy: # new highest energy shower
                curr_subleading_energy = curr_leading_energy
                curr_subleading_momentum = curr_leading_momentum
                curr_leading_energy = curr_particle_energy
                curr_leading_momentum = curr_particle_momentum
            elif curr_particle_energy > curr_subleading_energy: # new subleading shower
                curr_subleading_energy = curr_particle_energy
                curr_subleading_momentum = curr_particle_momentum
        
    curr_total_momentum = curr_leading_momentum + curr_subleading_momentum
    curr_total_momentum_magnitude = np.linalg.norm(curr_total_momentum)
    curr_total_momentum_beam_angle = np.arccos(curr_total_momentum[2] / curr_total_momentum_magnitude) * 180 / np.pi

    curr_opening_angle = np.arccos(np.dot(curr_leading_momentum, curr_subleading_momentum) / (np.linalg.norm(curr_leading_momentum) * np.linalg.norm(curr_subleading_momentum))) * 180 / np.pi

    leading_energies.append(curr_leading_energy)
    subleading_energies.append(curr_subleading_energy)
    opening_angles.append(curr_opening_angle)
    total_momentum_beam_angles.append(curr_total_momentum_beam_angle)

two_HE_shower_0_HE_track_df["leading_energy"] = leading_energies
two_HE_shower_0_HE_track_df["subleading_energy"] = subleading_energies
two_HE_shower_0_HE_track_df["opening_angle"] = opening_angles
two_HE_shower_0_HE_track_df["total_momentum_beam_angle"] = total_momentum_beam_angles

two_HE_shower_0_HE_track_df


In [ ]:
passing_two_HE_shower_0_HE_track_df = two_HE_shower_0_HE_track_df.query("PassOsc == True")
passing_two_HE_shower_0_HE_track_df

# Efficiencies

In [ ]:
plt.rcParams.update({'font.size': 16})

opening_angle_bins = np.linspace(0, 180, 18+1)
#opening_angle_bins = np.linspace(0, 90, 18+1)
#opening_angle_bins = np.linspace(0, 45, 16)
opening_angle_bin_centers = [(opening_angle_bins[i] + opening_angle_bins[i+1]) / 2 for i in range(len(opening_angle_bins) - 1)]

total_counts = np.histogram(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins)[0]
passing_counts = np.histogram(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins)[0]

effs = passing_counts / total_counts

eff_errs = np.sqrt(effs * (1 - effs) / total_counts)

plt.figure(figsize=(10, 7))
plt.hist(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", label="All Events")
plt.hist(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", label="Passing Events")
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("Events")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_threshold} MeV Showers, Zero >={track_threshold} MeV Tracks")
plt.legend()
plt.yscale("log")
plt.show()

plt.figure(figsize=(10, 7))
plt.hist(two_HE_shower_0_HE_track_df["opening_angle"], weights=two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", density=True, label="All Events")
plt.hist(passing_two_HE_shower_0_HE_track_df["opening_angle"], weights=passing_two_HE_shower_0_HE_track_df["Weight"], bins=opening_angle_bins, histtype="step", density=True, label="Passing Events")
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("Events (Normalized)")
plt.title(f"Events With Truth Requirements:\nTwo >={shower_threshold} MeV Showers, Zero >={track_threshold} MeV Tracks")
plt.legend()
plt.show()

plt.figure(figsize=(10, 7))
plt.errorbar(opening_angle_bin_centers, effs, yerr=eff_errs, fmt="o", capsize=5)
plt.xlabel("True Opening Angle (degrees)")
plt.ylabel("MiniBooNE nue CCQE Selection Efficiency")
plt.title(f"Predicted Events With Truth Requirements:\nTwo >={shower_threshold} MeV Showers, Zero >={track_threshold} MeV Tracks")
plt.show()

